# 1. Make predictions on a pre-trained model
## Key Points:
1. Data Loading and Preprocessing: Loads the CIFAR-10 dataset, normalizes the data, and one-hot encodes the labels.
2. Model Definition: Loads the pre-trained VGG16 model without the top layer and adds custom dense layers for CIFAR-10 classification.
3. Layer Freezing: Freezes all layers of the VGG16 base model to prevent them from being trained (no fine-tuning).
4. Model Compilation: Compiles the model with the Adam optimizer and categorical cross-entropy loss.
5. Training with Progress Bar: Trains the model with a progress bar for visual feedback.
6. Evaluation: Evaluates the model on the test dataset and prints out the performance metrics in a tabulated format.
* This code represents a minimalistic approach to transfer learning, where the pre-trained model is used as a feature extractor without any further training of its layers.

In [2]:
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
from tabulate import tabulate

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Load the VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze all layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers
x = base_model.output
x = Flatten()(x)
predictions = Dense(10, activation='softmax')(x)  # CIFAR-10 has 10 classes

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with a progress bar
num_epochs = 10
batch_size = 32

print("Training the model")
for epoch in range(num_epochs):
    with tqdm(total=len(x_train), desc=f"Epoch {epoch+1}/{num_epochs}", unit="samples") as pbar:
        history = model.fit(
            x_train, y_train,
            batch_size=batch_size,
            epochs=1,
            validation_data=(x_test, y_test),
            verbose=0,
            callbacks=[tf.keras.callbacks.LambdaCallback(on_batch_end=lambda batch, logs: pbar.update(batch_size))]
        )

# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

# Extract metrics from the history
train_loss = history.history['loss'][-1]
train_accuracy = history.history['accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_accuracy'][-1]

# Prepare data for tabulate
table_data = [
    ["Metric", "Value"],
    ["Train Loss", f"{train_loss:.4f}"],
    ["Train Accuracy", f"{train_accuracy:.4f}"],
    ["Validation Loss", f"{val_loss:.4f}"],
    ["Validation Accuracy", f"{val_accuracy:.4f}"],
    ["Test Loss", f"{loss:.4f}"],
    ["Test Accuracy", f"{accuracy:.4f}"]
]

# Print the table
print(tabulate(table_data, headers="firstrow", tablefmt="pretty"))

Training the model


Epoch 1/10: 50016samples [00:31, 1585.76samples/s]                      
Epoch 2/10: 50016samples [00:28, 1762.27samples/s]                      
Epoch 3/10: 50016samples [00:29, 1710.93samples/s]                      
Epoch 4/10: 50016samples [00:29, 1703.15samples/s]                      
Epoch 5/10: 50016samples [00:28, 1744.93samples/s]                      
Epoch 6/10: 50016samples [00:28, 1730.56samples/s]                      
Epoch 7/10: 50016samples [00:28, 1774.29samples/s]                      
Epoch 8/10: 50016samples [00:29, 1667.97samples/s]                      
Epoch 9/10: 50016samples [00:27, 1791.09samples/s]                      
Epoch 10/10: 50016samples [00:27, 1794.21samples/s]                      


+---------------------+--------+
|       Metric        | Value  |
+---------------------+--------+
|     Train Loss      | 1.1459 |
|   Train Accuracy    | 0.6045 |
|   Validation Loss   | 1.1990 |
| Validation Accuracy | 0.5856 |
|      Test Loss      | 1.1990 |
|    Test Accuracy    | 0.5856 |
+---------------------+--------+


In [3]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Make predictions on the test set
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

# Prepare data for tabulate
extra_table_data = [
    ["Precision", f"{precision:.4f}"],
    ["Recall", f"{recall:.4f}"],
    ["F1 Score", f"{f1:.4f}"]
]

# Print the updated table
print(tabulate(extra_table_data, headers="firstrow", tablefmt="pretty"))

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step
+-----------+--------+
| Precision | 0.5833 |
+-----------+--------+
|  Recall   | 0.5856 |
| F1 Score  | 0.5829 |
+-----------+--------+


# Fine-tune the pre-trained model
## Key Points:
1. **Data Loading and Preprocessing**: Loads the CIFAR-10 dataset, normalizes the data, and one-hot encodes the labels.
2. **Model Definition**: Loads the pre-trained VGG16 model without the top layer and adds custom dense layers for CIFAR-10 classification.
3. **Layer Freezing**: Initially freezes all layers of the VGG16 base model to prevent them from being trained during the initial training phase.
4. **Initial Training**: Trains the model with the base model layers frozen.
5. **Layer Unfreezing**: Unfreezes the last 4 layers of the base model for fine-tuning.
6. **Fine-Tuning Training**: Trains the model with some of the base model layers unfrozen.
7. **Evaluation**: Evaluates the model on the test dataset and prints out the performance metrics in a tabulated format.
* This approach allows the model to first learn the new task with the pre-trained weights and then fine-tune some of the deeper layers for potentially better performance.

In [5]:
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm
from tabulate import tabulate

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Load the VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add custom top layers
x = base_model.output
x = Flatten()(x)
predictions = Dense(10, activation='softmax')(x)  # CIFAR-10 has 10 classes

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model (initially with frozen base model layers)
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Initial training with frozen base model
initial_epochs = 5
batch_size = 32

print("Training the model (initially frozen)")
for epoch in range(initial_epochs):
    with tqdm(total=len(x_train), desc=f"Epoch {epoch+1}/{initial_epochs}", unit="samples") as pbar:
        history = model.fit(
            x_train, y_train,
            batch_size=batch_size,
            epochs=1,
            validation_data=(x_test, y_test),
            verbose=0,
            callbacks=[tf.keras.callbacks.LambdaCallback(on_batch_end=lambda batch, logs: pbar.update(batch_size))]
        )

# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-4:]:  # Unfreeze the last 4 layers
    layer.trainable = True

# Recompile the model for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tuning training
fine_tuning_epochs = 10

print("Fine-tuning the model")
for epoch in range(fine_tuning_epochs):
    with tqdm(total=len(x_train), desc=f"Epoch {epoch+1}/{fine_tuning_epochs}", unit=" samples") as pbar:
        history = model.fit(
            x_train, y_train,
            batch_size=batch_size,
            epochs=1,
            validation_data=(x_test, y_test),
            verbose=0,
            callbacks=[tf.keras.callbacks.LambdaCallback(on_batch_end=lambda batch, logs: pbar.update(batch_size))]
        )

# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

# Extract metrics from the history
train_loss = history.history['loss'][-1]
train_accuracy = history.history['accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_accuracy'][-1]

# Prepare data for tabulate
table_data = [
    ["Metric", "Value"],
    ["Train Loss", f"{train_loss:.4f}"],
    ["Train Accuracy", f"{train_accuracy:.4f}"],
    ["Validation Loss", f"{val_loss:.4f}"],
    ["Validation Accuracy", f"{val_accuracy:.4f}"],
    ["Test Loss", f"{loss:.4f}"],
    ["Test Accuracy", f"{accuracy:.4f}"]
]

# Print the table
print(tabulate(table_data, headers="firstrow", tablefmt="pretty"))

Training the model (initially frozen)


Epoch 1/5: 50016samples [00:29, 1724.65samples/s]                      
Epoch 2/5: 50016samples [00:28, 1762.14samples/s]                      
Epoch 3/5: 50016samples [00:31, 1598.86samples/s]                      
Epoch 4/5: 50016samples [00:31, 1594.49samples/s]                      
Epoch 5/5: 50016samples [00:30, 1630.63samples/s]                      


Fine-tuning the model


Epoch 1/10: 50016 samples [00:59, 842.24 samples/s]                     
Epoch 2/10: 50016 samples [00:57, 868.37 samples/s]                     
Epoch 3/10: 50016 samples [00:57, 868.48 samples/s]                     
Epoch 4/10: 50016 samples [00:57, 865.52 samples/s]                     
Epoch 5/10: 50016 samples [00:57, 867.71 samples/s]                     
Epoch 6/10: 50016 samples [00:58, 860.24 samples/s]                     
Epoch 7/10: 50016 samples [00:57, 870.50 samples/s]                     
Epoch 8/10: 50016 samples [00:57, 869.06 samples/s]                     
Epoch 9/10: 50016 samples [00:57, 868.20 samples/s]                     
Epoch 10/10: 50016 samples [00:57, 869.66 samples/s]                     


+---------------------+--------+
|       Metric        | Value  |
+---------------------+--------+
|     Train Loss      | 0.2578 |
|   Train Accuracy    | 0.9212 |
|   Validation Loss   | 0.7874 |
| Validation Accuracy | 0.7468 |
|      Test Loss      | 0.7874 |
|    Test Accuracy    | 0.7468 |
+---------------------+--------+


In [6]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Make predictions on the test set
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

# Prepare data for tabulate
extra_table_data = [
    ["Precision", f"{precision:.4f}"],
    ["Recall", f"{recall:.4f}"],
    ["F1 Score", f"{f1:.4f}"]
]

# Print the updated table
print(tabulate(extra_table_data, headers="firstrow", tablefmt="pretty"))


313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
+-----------+--------+
| Precision | 0.7450 |
+-----------+--------+
|  Recall   | 0.7468 |
| F1 Score  | 0.7450 |
+-----------+--------+
